# Units in Python

In [ ]:
import numpy as np

### Find the position (x) of a rocket moving at a constant velocity (v) after a time (t)

<img src="https://uwashington-astro300.github.io/A300_images/rocket.png" width="400"/>

In [ ]:
def find_position(velocity, time):
    result = velocity * time
    return result

### If v = 10 m/s and t = 10 s

In [ ]:
my_velocity = 10
my_time = 10

find_position(my_velocity, my_time)

### No problem, x = 100 m

---

### Now v = 10 mph and t = 10 minutes

In [ ]:
my_other_velocity = 10
my_other_time = 10

find_position(my_other_velocity, my_other_time)

### x = 100 miles minutes / hour ??

---
# The Astropy Units package to the rescue

In [ ]:
from astropy import units as u
from astropy import constants as const
from astropy.units import imperial
imperial.enable()

#### *Note: because we imported the `units` package as `u`, you cannot use **u** as a variable name.*

---

### Add units to values using `u.UNIT` where UNIT is an [Astropy Unit](http://docs.astropy.org/en/stable/units/index.html#module-astropy.units.si)

* To add a UNIT to a VALUE you multiply (*) the VALUE by the UNIT
* You can make compound units like: `u.m / u.s`

In [ ]:
my_velocity = 10 * (u.m / u.s)
my_time = 10 * u.s

In [ ]:
def find_position(velocity, time):
    result = velocity * time
    return result

In [ ]:
find_position(my_velocity, my_time)

#### Notice the difference when using imperial units - (`imperial.UNIT`)

In [ ]:
my_other_velocity = 10.0 * (imperial.mi / u.h)
my_other_time = 10 * u.min

In [ ]:
find_position(my_other_velocity, my_other_time)

### Notice that the units are a bit strange. We can simplify this using `.decompose()`

* Default to SI units

In [ ]:
find_position(my_other_velocity, my_other_time).decompose()

### I like to put the `.decompose()` in the return of the function:

In [ ]:
def find_position(velocity, time):
    result = velocity * time
    return result.decompose()

In [ ]:
find_position(my_other_velocity, my_other_time)

In [ ]:
find_position(1 * (imperial.fur / u.fortnight), 1 * u.wk)

### Unit conversion is really easy!

In [ ]:
rocket_position = find_position(my_other_velocity, my_other_time)

rocket_position

In [ ]:
rocket_position.to(u.km)

In [ ]:
rocket_position.to(imperial.mi)

In [ ]:
rocket_position.si                     # quick conversion to SI units

In [ ]:
rocket_position.cgs                    # quick conversion to CGS units

## It is always better to do unit conversions **outside** of functions

### Be careful adding units to something that already has units!

* `velocity` and `time` have units.
* By doing `result * u.km` you are adding another unit

In [ ]:
def find_position_wrong(velocity, time):
    result = velocity * time
    
    return (result * u.km).decompose()

In [ ]:
find_position_wrong(my_other_velocity, my_other_time)

----
## Unit equivalencies

In many fields there are very common unit conversions that do not seem to make sense based on the units involved.

For example in astronomy, the distances to stars can be based on the parallax angle. Parallax is an angle and distance is a length, 
If you do not know the context of the convesion, it does not seem to be correct.

The Astropy units package has a command called `equivalencies` that allow you to do unit conversions under certain physical assumptions.


### Parallax -> Distance

In [ ]:
my_parallax = 34.26 * u.mas

my_parallax

In [ ]:
# This will not work!

my_parallax.to(u.parsec)

### With `equivalencies` we can convert it to **any** length

In [ ]:
my_parallax.to(u.parsec, equivalencies=u.parallax())

In [ ]:
my_parallax.to(u.lyr, equivalencies=u.parallax())

In [ ]:
my_parallax.to(u.AU, equivalencies=u.parallax())

### You can find the units in `Astropy` that are of the same type with `.find_equivalent_units()`

In [ ]:
(u.pc).find_equivalent_units()

### Wavelength -> Frequency or Energy

In [ ]:
my_wavelength = 1000 * u.nm

my_wavelength

In [ ]:
my_wavelength.to(u.Hz, equivalencies=u.spectral())

In [ ]:
my_wavelength.to(u.eV, equivalencies=u.spectral())

---
### You do not have to worry about working in different units (**as long as they are the same type**)!

* No conversions needed
* Just make sure you assign units

In [ ]:
my_velocity

In [ ]:
my_other_velocity

In [ ]:
my_velocity + my_other_velocity

#### Units default to SI units

In [ ]:
my_time, my_other_time

In [ ]:
my_time + my_other_time

---
### Be careful combining quantities with different units!

In [ ]:
my_velocity + my_time

In [ ]:
2 + my_time

---

## Dimentionless Units

In [ ]:
dimless_y = 10 * u.dimensionless_unscaled

dimless_y

In [ ]:
dimless_y.unit

In [ ]:
dimless_y.decompose()   # returns the scale of the dimentionless quanity

### You can also make dimentionless units by dividing two quantities with the same type of unit.

In [ ]:
my_velocity, my_other_velocity

In [ ]:
my_new_velocity = my_velocity / my_other_velocity

In [ ]:
my_new_velocity.decompose()

### Some math functions only make sense with dimentionless quanities

In [ ]:
np.log(2 * u.m)

In [ ]:
np.log(2 * u.dimensionless_unscaled)

### Or they expect the correct type of unit!

In [ ]:
np.sin(2 * u.m)

In [ ]:
np.sin(2 * u.deg)

## Using units can save you headaches. 

* All of the trig functions expect all angles to be in radians. 
* If you forget this, it can lead to problems that are hard to debug

$$ \large
\sin(90^{\circ}) + \sin(45^{\circ}) =  1 + \frac{\sqrt{2}}{2} \approx 1.7071
$$

In [ ]:
np.sin(90) + np.sin(45)

In [ ]:
np.sin(90 * u.deg) + np.sin(45 * u.deg)

---

## You can define your own units

In [ ]:
ringo = u.def_unit('Ringos', 3.712 * imperial.yd)

In [ ]:
rocket_position.to(ringo)

In [ ]:
my_velocity.to(ringo / u.s)

#### ...Since `ringo` is self-defined it does not have a `u.` in front of it

### You can access the number and unit part of the Quantity separately:

In [ ]:
my_velocity.value

In [ ]:
my_velocity.unit

### This is useful in formatting output:

In [ ]:
f"The velocity of the first particle is {my_velocity.value:.1f} in the units of {my_velocity.unit:s}."

---

# Constants

The `Astropy` package also includes a whole bunch of built-in constants to make your life easier.

* The package is usually imported as `const`

### [Astropy Constants](http://docs.astropy.org/en/stable/constants/index.html#reference-api)

In [ ]:
const.G

In [ ]:
const.M_sun

---

### An Example: The velocity of an object in circular orbit around the Sun is

$$\large
v=\sqrt{GM_{\odot}\over d} 
$$

### What is the velocity of an object at 1 AU from the Sun?

In [ ]:
def find_orbit_v(distance):
    result = np.sqrt(const.G * const.M_sun / distance)
    return result.decompose()

In [ ]:
my_distance = 1 * u.pc

In [ ]:
orbit_v = find_orbit_v(my_distance)
orbit_v

In [ ]:
orbit_v.to(u.km/u.s)

In [ ]:
orbit_v.to(ringo/u.ms)

### Be careful about the difference between a unit and a constant

In [ ]:
my_star = 1 * u.solMass
my_star

In [ ]:
my_star.unit

In [ ]:
const.M_sun

In [ ]:
const.M_sun.unit

----

# Astropy QTables are units-aware

In [ ]:
from astropy.table import QTable
from astroquery.gaia import Gaia

In [ ]:
query_one = """
SELECT TOP 10
source_id, parallax
FROM gaiadr3.gaia_source_lite
WHERE parallax > 200
ORDER BY parallax DESC
"""

In [ ]:
my_job_query = Gaia.launch_job(query_one)

In [ ]:
print(my_job_query)

In [ ]:
my_parallax_table = my_job_query.get_results()

my_parallax_table

## Adding a `Distance` column is now trivial...

In [ ]:
my_parallax_table['Distance'] = my_parallax_table['parallax'].to(u.parsec, equivalencies=u.parallax())

In [ ]:
my_parallax_table

In [ ]:
my_parallax_table['Distance'].to(u.lyr)

### Can save `QTables` with all the units info intact (`.ecsv`).

In [ ]:
my_parallax_table.write('./Data/Star_QTable.ecsv', 
                        format='ascii.ecsv', 
                        overwrite=True)

In [ ]:
my_new_table = QTable.read('./Data/Star_QTable.ecsv', format='ascii.ecsv')

In [ ]:
my_new_table

---
# ⭐ Bonus Content ⭐

## [Mars Climate Orbiter](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter)

Aerobraking is a spaceflight maneuver uses the drag of flying a spacecraft through the (upper) atmosphere of a world to slow a spacecraft and lower its orbit. Aerobraking requires way less fuel compared to using propulsion to slow down.



On September 8, 1999, Trajectory Correction Maneuver-4 (TCM-4) was computed to place the Mars Climate Orbiter spacecraft at an optimal position for an orbital insertion maneuver that would bring the spacecraft around Mars at an altitude of 226 km. At this altitude the orbiter would skim through Mars' upper atmosphere, gradually aerobraking for weeks.

The calculation of TCM-4 was done in United States imperical units. The software that calculated the total needed impulse of the thruster firing produced results in pound-force seconds.

### Mars Climate Orbiter

* Mass = 338 kg (745 lbs)
* ΔV needed for TCM-4 = 9.2 m/s (30.2 fps)
* Need to calculate Impulse

### Impulse is a change in momentum

$$ \Large 
I = \Delta\ p\ =\ m\Delta v 
$$

#### Impulse calculated in imperial units:

In [ ]:
imperial_impulse = (745 * (imperial.lb)) *  (30.2 * (imperial.ft / u.s))

imperial_impulse.to(imperial.lbf * u.s)

The computed impulse value was then sent to the spacecraft and was used to fire the thuster on September 15, 1999. The computer that fired the thuster expected the impulse to be in SI units (newton-seconds). SI units are required by NASA's Software Interface Specification (SIS).

In [ ]:
imperial_impulse.to(u.N * u.s)

#### $\Delta$v that would be the result of an impuse of 669.3 (N * s) for M = 338 kg:

In [ ]:
bad_deltav = (669.3 * (u.N * u.s)) / (338 * (u.kg))

bad_deltav.decompose()

#### What the $\Delta$v should have been for M = 338 kg:

In [ ]:
good_deltav = (3110.6 * (u.N * u.s)) / (338 * (u.kg))

good_deltav.decompose()

This $\Delta$v was way too small! At this speed the spacecraft's trajectory would have taken it within 57 km (35 miles) of the surface. At this altitude, the spacecraft would likely have skipped violently off the denser-than-expected atmosphere, and it was either destroyed in the atmosphere, or re-entered heliocentric space.

<img src="https://uwashington-astro300.github.io/A300_images/MCO_Orbit.png" width="700"/>